In [1]:
# can add some more maybe
# Some code from https://github.com/dhimmel/SIDER4/blob/2acca0b065e736bc99702906024efd4718e502ee/SIDER4.ipynb

In [1]:
import os
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import parse_xml
from parse_xml import *
from lxml import etree
import xml.etree.ElementTree as ET
os.system('wget -N -P input/ http://sideeffects.embl.de/media/download/meddra_all_se.tsv.gz')
os.system('gunzip input/meddra_all_se.tsv.gz')

--2023-03-22 22:30:21--  http://sideeffects.embl.de/media/download/meddra_all_se.tsv.gz
Resolving sideeffects.embl.de (sideeffects.embl.de)... 194.94.44.34
Connecting to sideeffects.embl.de (sideeffects.embl.de)|194.94.44.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2381171 (2.3M) [application/x-gzip]
Saving to: ‘input/meddra_all_se.tsv.gz’

     0K .......... .......... .......... .......... ..........  2%  129K 18s
    50K .......... .......... .......... .......... ..........  4%  130K 17s
   100K .......... .......... .......... .......... ..........  6%  259K 14s
   150K .......... .......... .......... .......... ..........  8% 1.12M 11s
   200K .......... .......... .......... .......... .......... 10%  260K 10s
   250K .......... .......... .......... .......... .......... 12%  329K 9s
   300K .......... .......... .......... .......... .......... 15%  259K 9s
   350K .......... .......... .......... .......... .......... 17%  259K 8s
   400K ....

512

## DrugBank -IS- PubChem

In [9]:
db2pubchem = json.load(open('output/compound2compound/db2pubchemsub.json'))
pubchem2db = json.load(open('output/compound2compound/pubchemcomp2db.json'))
print(len(pubchem2db), 'PubChem Substance to DrugBank')
print(len(db2pubchem), 'DrugBank to Pubchem Substance')

8725 PubChem Substance to DrugBank
10452 DrugBank to Pubchem Substance


## PubChem (compound) to MeSH (side effect)

In [10]:
umls2mesh = json.load(open('output/otherMappings/umls2mesh.json'))

In [11]:
pubchem2side = dict()
# 1 & 2: STITCH compound ids (flat/stereo, see above)
# 3: UMLS concept id as it was found on the label
# 4: MedDRA concept type (LLT = lowest level term, PT = preferred term; in a few cases the term is neither LLT nor PT)
# 5: UMLS concept id for MedDRA term
# 6: side effect name
#All side effects found on the labels are given as LLT. Additionally, the PT is shown. There is at least one
#PT for every LLT, but sometimes the PT is the same as the LLT.


with open('input/meddra_all_se.tsv') as fin:
    for i,line in enumerate(fin):
        line = line.split('\t')
        stitch_sterio = line[1] # Hetionet did stereo https://github.com/dhimmel/SIDER4/blob/2acca0b065e736bc99702906024efd4718e502ee/SIDER4.ipynb
        pubchem = stitch_sterio[3:].strip('0')
        side = line[2]
        meddra_type = line[3]
        side2 = line[4]
        if meddra_type == 'PT':   # Only reduces side effects from 146,518 to 145,617. 1,550 drugs either way.
            try: mesh_sides = umls2mesh[side]
            except: continue
            for mesh_side in mesh_sides:
                pubchem2side.setdefault(pubchem,set()).add(mesh_side)

for pc,se in pubchem2side.items():
    pubchem2side[pc] = list(se)        

## DrugBank -causes-> MeSH Side Effect

In [12]:
pubchems = list(pubchem2db.keys())
sider_pc = list(pubchem2side.keys())
db2se = dict()
i = 0
for pc in pubchems:
    if pc in sider_pc:
        ses = pubchem2side[pc]
        dbs = pubchem2db[pc]
        for db in dbs:
            for se in ses:
                db2se.setdefault(db,[]).append(se)
                #writer.writerow([db,se,'-causes_side_effect->'])

In [13]:
dbs = list(db2se.keys())
print('Drugs', len(dbs))
se = list(db2se.values())
se1 = []
for s in se:
    se1+=s 

se2 = set()
for s in se:
    se2 = se2.union(set(s))
print('Side Effects',len(se2))
print('Edges',len(se1))

Drugs 903
Side Effects 1800
Edges 66819


In [14]:
pubchems = list(pubchem2db.keys())
sider_pc = list(pubchem2side.keys())

with open('output/compound2disease/edges_drugbank2meshsideeffect.csv','w') as fout:
    writer = csv.writer(fout) 
    writer.writerow(['Compound (DrugBank)','Disease (MeSH)','Relationship'])
    db2se = dict()
    for pc in pubchems:
        if pc in sider_pc:
            
            ses = pubchem2side[pc] # Side effects
            dbs = pubchem2db[pc]        # DrugBank IDs
            
            for db in dbs:
                for se in ses:
                    db2se.setdefault(db,[]).append(se)
                    writer.writerow(['DrugBank_Compound:'+db,'MeSH_Disease:'+se,'-causes_side_effect->'])
!cp 'output/compound2disease/edges_drugbank2meshsideeffect.csv' 'output/edges/edges_drugbank2meshsideeffect.csv'

In [19]:
file = 'Compound_(DrugBank)_sideeffect_Disease_(MeSH).csv'
df = pd.read_csv('output/edges/edges_drugbank2meshsideeffect.csv')
df.to_csv(os.path.join('output/edges to use', file), index=False)
df

,Compound (DrugBank),Disease (MeSH),Relationship
0,DrugBank_Compound:DB00006,MeSH_Disease:D001281,-causes_side_effect->
1,DrugBank_Compound:DB00006,MeSH_Disease:D006417,-causes_side_effect->
2,DrugBank_Compound:DB00006,MeSH_Disease:D006471,-causes_side_effect->
3,DrugBank_Compound:DB00006,MeSH_Disease:D014570,-causes_side_effect->
4,DrugBank_Compound:DB00006,MeSH_Disease:D004844,-causes_side_effect->
...,...,...,...
66814,DrugBank_Compound:DB15566,MeSH_Disease:D008599,-causes_side_effect->
66815,DrugBank_Compound:DB15566,MeSH_Disease:D008581,-causes_side_effect->
66816,DrugBank_Compound:DB15566,MeSH_Disease:D000799,-causes_side_effect->
66817,DrugBank_Compound:DB15566,MeSH_Disease:D006332,-causes_side_effect->


## MeSH Compound -> Side Effect

In [18]:
db2mesh = json.load(open('output/compound2compound/db2mesh.json'))

In [20]:
pubchems = list(pubchem2db.keys())
sider_pc = list(pubchem2side.keys())

with open('output/compound2disease/edge_meshcompound2sideeffect.csv','w') as fout:
    writer = csv.writer(fout) 
    writer.writerow(['Compound (MeSH)','Disease (MeSH)','Relationship'])
    mesh2se = dict()
    for pc in pubchems:
        if pc in sider_pc:
            
            ses = pubchem2side[pc] # Side effects
            dbs = pubchem2db[pc]        # DrugBank IDs
            
            # DrugBank Drugs
            for db in dbs:
                
                # MeSH Drugs
                try: meshes = db2mesh[db]
                except: continue
                for mesh in meshes:

                    # Side Effect
                    for se in ses:
                        mesh2se.setdefault(mesh,[]).append(se)
                        writer.writerow(['MeSH_Compound:'+mesh,'MeSH_Disease:'+se,'-causes_side_effect->'])
                        
! cp 'output/compound2disease/edge_meshcompound2sideeffect.csv' 'output/edges/edge_meshcompound2sideeffect.csv'

file = 'Compound_(DrugBank)_sideeffect_Disease_(MeSH).csv'
df = pd.read_csv('output/edges/edge_meshcompound2sideeffect.csv')
df.to_csv(os.path.join('output/edges to use', file), index=False)
df

,Compound (MeSH),Disease (MeSH),Relationship
0,MeSH_Compound:C074619,MeSH_Disease:D001281,-causes_side_effect->
1,MeSH_Compound:C074619,MeSH_Disease:D006417,-causes_side_effect->
2,MeSH_Compound:C074619,MeSH_Disease:D006471,-causes_side_effect->
3,MeSH_Compound:C074619,MeSH_Disease:D014570,-causes_side_effect->
4,MeSH_Compound:C074619,MeSH_Disease:D004844,-causes_side_effect->
...,...,...,...
69253,MeSH_Compound:C009935,MeSH_Disease:D008599,-causes_side_effect->
69254,MeSH_Compound:C009935,MeSH_Disease:D008581,-causes_side_effect->
69255,MeSH_Compound:C009935,MeSH_Disease:D000799,-causes_side_effect->
69256,MeSH_Compound:C009935,MeSH_Disease:D006332,-causes_side_effect->
